# Final assigment
## 0. Table of contents
* [Introduction](#introduction)
* [Methodology](#methodology)
* [Data](#data)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

# 1. Data <a name="data"></a>
## 1.1. Houston coordinate:
The latitude of Houston, TX, USA is 29.749907, and the longitude is -95.358421. 
ref https://www.latlong.net/place/houston-tx-usa-27227.html
## 1.2. Nearby venue information
Nearby venue information are provided by FourSquare API
www.foursquare.com
## 1.3. House prices per ft²

## 1.4. Criminality data

# 2. Methodology <a name="methodology"></a>
## 1.  Clustering - finding region with similitude between origin and destination
Houston, the destination town chosen, will be split in 10x10 miles square  
Each square will be analyzed according to avaiable foursquare data.  
Then, each square will be clusterized using K-means clustering technics 
Then, the origin town neighborood will be assigned a cluster using the same model.
A map showing location having the most similitude with the origin will be shown using geopy***
## 2. Incoporetagin criminality and housing price   
Then, a pareto optimization will be perform to take into account price and criminiality for each destination cells.  
An updated map will be presented showing similarcells in destination town to origin that are deemed optimum in regards to the criminality and house pricing (per ft²)



# 3. Assumption  
- No taking into account road/traffic/public transportation
- No taking into account a specific work location, anywhere is just as good in that matter


# 4. Algorithm
## 4.0 Table of contents
- 4.1 [Librairies import](#401)
* [4.2 Houston region clustering](#402)
* [4.3 Liege region clustering](#403)
* Houston, best destination definition
* Adding criminality and house prices 

## 1. Importing librairies and ressources <a name="401"></a>

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes 
# uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
# 
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# 
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# 
# import k-means from clustering stage
from sklearn.cluster import KMeans
# 
# !conda install -c conda-forge folium=0.5.0 --yes 
# uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

import time # in order to use sleep functions to avoid timeout of some API
print('Libraries imported.')

Libraries imported.


## 2. Houston region clustering <a name="402"></a>
### 2.1 Create a dataset of region with there related coordinate 
Houston have a area of ~ 1800 km². That area is equivalent of a square of 42km on 42km. We will split the whole region by 100 square subregions, called cell here after of 4km side

In [3]:
# variables
# # region diameter
Number_of_Cell = 100
Cell_Diam = 4 # km
Houston_Coord_lat = 29.749907
Houston_Coord_lon = -95.358421

We have to find the coordinate of each cell. We will use UTM package to transform coordinate to UTM projection, shit the projection by the defined distance between two cells and convert back to coordinate.
More info there:
https://stackoverflow.com/questions/16266809/convert-from-latitude-longitude-to-x-y

We will also sligthly shift the center so all cells are contained inside the Belt 8, surounding Houston 


In [4]:
# value are given in meter from a reference points
#!conda install -c conda-forge utm 
import utm
UTM_Center_Houston = utm.from_latlon(Houston_Coord_lat, Houston_Coord_lon)
UTM_Center_Houston_Easting = UTM_Center_Houston[0]
UTM_Center_Houston_Norting = UTM_Center_Houston[1]

# loop for each cell
index = 0
Houston_Cell = []
for i in range(-5, 5):
    for j in range(-4, 6): # note, this has been offset to remains in Houston boundaries
        UTM_Easting = UTM_Center_Houston_Easting + i * Cell_Diam * 1000 # convert from km to m 
        UTM_Norting = UTM_Center_Houston_Norting + j * Cell_Diam * 1000
        lat, lon = utm.to_latlon(UTM_Easting, UTM_Norting, UTM_Center_Houston[2], UTM_Center_Houston[3])
        Houston_Cell.append({'Cell_index' : index, 'Cell_Latitude': lat, 'Cell_Longitude': lon})
        index +=1

pd_Houston_Cell = pd.DataFrame(Houston_Cell)
pd_Houston_Cell.head(5)


,Cell_Latitude,Cell_Longitude,Cell_index
0,29.601804,-95.561416,0
1,29.637868,-95.562329,1
2,29.673933,-95.563244,2
3,29.709996,-95.564160,3
4,29.746060,-95.565077,4


In [445]:
# check the resutls on the map:
# create map
map_clusters = folium.Map(location=[Houston_Coord_lat, Houston_Coord_lon], zoom_start=11)

# add markers to the map
markers_colors = []
for lat, lon, poi in zip(pd_Houston_Cell['Cell_Latitude'], pd_Houston_Cell['Cell_Longitude'], pd_Houston_Cell['Cell_index']):
    label = folium.Popup(str(poi), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        fill=True,
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 2.2 Get venues for each cells in a specific range
We will now use Foursquare API to get all venue in a certain range for each cell.
We will first focus on venues in a 15 km range and create a data set out of it
### 2.2.3 Load Foursquare credentials

In [6]:
secrets = json.load(open('4S_secret.json'))
CLIENT_ID = secrets['CLIENT_ID']
CLIENT_SECRET = secrets['CLIENT_SECRET']
VERSION = secrets['VERSION']
print VERSION

print('Secret Credential loaded')

20180605
Secret Credential loaded


### 2.2.4 create function to extract category of the venues

In [7]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

### 2.2.5 Get all venue for each cells in a certain radius

In [9]:
radius = 1500
LIMIT = 100
Results_list = list()
for i in range (0, len(pd_Houston_Cell.index)): # 2): # 
    latitude = pd_Houston_Cell.iloc[i]['Cell_Latitude']
    longitude = pd_Houston_Cell.iloc[i]['Cell_Longitude']
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
    Results_list.append(requests.get(url).json())
    
#print Results_list

### 2.2.6. put all results in a nice dataframe containing Cells index, and venues

In [10]:
#tobecontinued try to get a dataframe like in coursera 
nearby_venues_list = list()
for results, index, lat, lon in zip(Results_list, pd_Houston_Cell.index, pd_Houston_Cell.Cell_Latitude, pd_Houston_Cell.Cell_Longitude):
    venues = results['response']['groups'][0]['items']
    # print ("venue", venues)
    nearby_venues = json_normalize(venues) # flatten JSON
    # print ('nearby_venues', nearby_venues)
    # filter columns
    filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
    # print 'toto'
    # print nearby_venues
    # check if results dataframe is empty 
    if nearby_venues.empty:
        print 'empty'
    else:
        nearby_venues =nearby_venues.loc[:, filtered_columns]

        # filter the category for each row
        nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

        # clean columns
        nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
        index = [index] * len(nearby_venues.index)
        nearby_venues['Cell_index'] = index
        nearby_venues['Cell_Latitude'] = lat
        nearby_venues['Cell_Longitude'] = lon
    # Store results 
    nearby_venues_list.append(nearby_venues)
nearby_venues_list[0]

# let's group everything in one dataframe instead of a list of dataframe
df_Houston_Cell_Venue = pd.DataFrame() # nearby_venues_list[0]
for nv in nearby_venues_list:
    df_Houston_Cell_Venue = df_Houston_Cell_Venue.append(nv)

# reset index
df_Houston_Cell_Venue.reset_index(drop =True, inplace=True)


# lets reorganize the dataframe
#print nearby_venues_df['Postcode']
df_Houston_Cell_Venue = df_Houston_Cell_Venue.rename(columns={
    "name": "Venue",
    "lat": "Venue_Latitude",
    "lng": "Venue_Longitude",
    "categories": "Venue_Category"
})

# let's reorder 
df_Houston_Cell_Venue = df_Houston_Cell_Venue[['Cell_index', 'Cell_Latitude', 'Cell_Longitude', 'Venue', 'Venue_Category',
                                     'Venue_Latitude', 'Venue_Longitude']]




df_Houston_Cell_Venue.head(5)


empty


,Cell_index,Cell_Latitude,Cell_Longitude,Venue,Venue_Category,Venue_Latitude,Venue_Longitude
0,0,29.601804,-95.561416,Walgreens,Pharmacy,29.600119,-95.563614
1,0,29.601804,-95.561416,Starbucks,Coffee Shop,29.599570,-95.563985
2,0,29.601804,-95.561416,Tornado Burger,Burger Joint,29.611505,-95.564204
3,0,29.601804,-95.561416,El Vaquero,Mexican Restaurant,29.589083,-95.564442
4,0,29.601804,-95.561416,CVS pharmacy,Pharmacy,29.600460,-95.565012


## 2.3 Analyze Houston cells according to venues nearby
### 2.3.1 Refactor dataframe so it can be analyzed

In [32]:
# one hot encoding
df_Houston_Cell_Venue_onehot = pd.get_dummies(df_Houston_Cell_Venue[['Venue_Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
df_Houston_Cell_Venue_onehot['Cell_index'] = df_Houston_Cell_Venue['Cell_index'] 

# move neighborhood column to the first column
col_list =  list(df_Houston_Cell_Venue_onehot.columns)
col_list.pop(col_list.index('Cell_index'))
fixed_columns = ['Cell_index'] + col_list

df_Houston_Cell_Venue_onehot = df_Houston_Cell_Venue_onehot[fixed_columns]

print df_Houston_Cell_Venue_onehot.shape
df_Houston_Cell_Venue_onehot.head()

# pd.options.display.max_rows=1000
# print df_Houston_Cell_Venue_onehot
# 
# pd.options.display.max_rows=12

(3218, 331)


,Cell_index,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,...,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo,Zoo Exhibit
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2.3.2 Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [37]:
df_Houston_Cell_Venue_grouped = df_Houston_Cell_Venue_onehot.groupby('Cell_index').mean().reset_index()


print df_Houston_Cell_Venue_grouped.shape
df_Houston_Cell_Venue_grouped.head()

(99, 331)


,Cell_index,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,...,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo,Zoo Exhibit
0,0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.025641,0.000000,0.00,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.021739,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.021739,0.00,0.0,0.0,0.0
3,3,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.01,0.0,0.0,0.0
4,4,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.0,0.0,0.0


### 2.3.3 Let's create a dataframe containing the most common venue by neighborhood

In [13]:
# reusing ibm courses function (DP0701EN-3-3-2-Neighborhoods-New-York-py-v1.0)
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Cell_index']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
df_Houston_Cell_venues_sorted = pd.DataFrame(columns=columns)
df_Houston_Cell_venues_sorted['Cell_index'] = df_Houston_Cell_Venue_grouped['Cell_index']

for ind in np.arange(df_Houston_Cell_Venue_grouped.shape[0]):
    df_Houston_Cell_venues_sorted.iloc[ind, 1:] = return_most_common_venues(df_Houston_Cell_Venue_grouped.iloc[ind, :], num_top_venues)

df_Houston_Cell_venues_sorted.head(5)

,Cell_index,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Mexican Restaurant,Pharmacy,Coffee Shop,Storage Facility,Fast Food Restaurant,Burger Joint,Camera Store,Gas Station,Sandwich Place,Juice Bar
1,1,Mexican Restaurant,Sandwich Place,Seafood Restaurant,Deli / Bodega,Video Store,Bar,Comfort Food Restaurant,Pharmacy,Fried Chicken Joint,Food
2,2,Fast Food Restaurant,Mobile Phone Shop,Mexican Restaurant,Gas Station,Video Store,Asian Restaurant,Storage Facility,Other Repair Shop,Fried Chicken Joint,Motel
3,3,Chinese Restaurant,Asian Restaurant,Bubble Tea Shop,Vietnamese Restaurant,Korean Restaurant,Bakery,Hotel,Ice Cream Shop,Juice Bar,Fast Food Restaurant
4,4,Hotel,Sandwich Place,Grocery Store,Bakery,Chinese Restaurant,Mexican Restaurant,Pizza Place,Discount Store,Japanese Restaurant,Breakfast Spot


### 2.3.4 Cluster neighborood
We'll use K-means method to clusterize Neighborhood.
We'll use the dataframe containing all venue type frequencies
We choose ten cluster (1/10 of total sample) in order to have a quality split  
- a couple of isaloted and very specific neigbhorood  
- a good seperation of hte remaining cluster : not all in one cluster and not one category by cluster

In [14]:
# number of cluster, K
K = 10

df_Houston_Cell_clustering = df_Houston_Cell_Venue_grouped.drop('Cell_index', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=K, random_state=0).fit(df_Houston_Cell_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:100] 

array([2, 1, 2, 1, 1, 1, 6, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1,
       1, 1, 1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 0, 2, 1, 1, 1, 1, 1,
       1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 3, 6, 2, 2, 2, 2,
       2, 2, 2, 1, 2, 1, 2, 2, 1, 2, 2, 5, 7, 2, 2, 2, 2, 1, 0, 0, 0, 4,
       2, 1, 2, 2, 2, 9, 2, 8, 2, 2, 1])

In [39]:
# now merge the results of the clustering to our dataframe containing the 10 most common venue
# add clustering labels
if 'Cluster Labels' in df_Houston_Cell_venues_sorted:
    df_Houston_Cell_venues_sorted['Cluster Labels'] = kmeans.labels_
else:
    df_Houston_Cell_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
df_Houston_Cell_venues_sorted['Cluster Labels'] = df_Houston_Cell_venues_sorted['Cluster Labels'].astype(int, inplace=True)
print df_Houston_Cell_venues_sorted['Cluster Labels'].dtype
df_Houston_Cell_venues_merged = pd_Houston_Cell


# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
#print df_Houston_Cell_venues_merged.head()
#print df_Houston_Cell_venues_sorted.head()

df_Houston_Cell_venues_merged = df_Houston_Cell_venues_merged.join(df_Houston_Cell_venues_sorted.set_index('Cell_index'), on='Cell_index')

# remove nan row 
# It happens when a cells had no nearby venues
df_Houston_Cell_venues_merged.dropna(axis=0, inplace=True) 


# force Cluster label to be int
df_Houston_Cell_venues_merged['Cluster Labels'] = df_Houston_Cell_venues_merged['Cluster Labels'].astype(int, inplace=True)

# show final cluster
df_Houston_Cell_venues_merged.head() 

int32


,Cell_Latitude,Cell_Longitude,Cell_index,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,29.601804,-95.561416,0,2,Mexican Restaurant,Pharmacy,Coffee Shop,Storage Facility,Fast Food Restaurant,Burger Joint,Camera Store,Gas Station,Sandwich Place,Juice Bar
1,29.637868,-95.562329,1,1,Mexican Restaurant,Sandwich Place,Seafood Restaurant,Deli / Bodega,Video Store,Bar,Comfort Food Restaurant,Pharmacy,Fried Chicken Joint,Food
2,29.673933,-95.563244,2,2,Fast Food Restaurant,Mobile Phone Shop,Mexican Restaurant,Gas Station,Video Store,Asian Restaurant,Storage Facility,Other Repair Shop,Fried Chicken Joint,Motel
3,29.709996,-95.564160,3,1,Chinese Restaurant,Asian Restaurant,Bubble Tea Shop,Vietnamese Restaurant,Korean Restaurant,Bakery,Hotel,Ice Cream Shop,Juice Bar,Fast Food Restaurant
4,29.746060,-95.565077,4,1,Hotel,Sandwich Place,Grocery Store,Bakery,Chinese Restaurant,Mexican Restaurant,Pizza Place,Discount Store,Japanese Restaurant,Breakfast Spot


### 2.3.5 Map the different cluster

In [446]:
# create map
map_clusters = folium.Map(location=[Houston_Coord_lat, Houston_Coord_lon], zoom_start=11)


# set color scheme for the clusters
x = np.arange(K)
ys = [i + x + (i*x)**2 for i in range(K)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_Houston_Cell_venues_merged['Cell_Latitude'], df_Houston_Cell_venues_merged['Cell_Longitude'], df_Houston_Cell_venues_merged['Cell_index'], df_Houston_Cell_venues_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# other thing to do
Analyse what is the main difference between each cluster


## 4.3 Liege region clustering <a name="403"></a>
Now that we have a model to classify Houston cells, we will classify also as well the origin neighborhood, in Liege and see to what kind of destination cells it is the most similar

In [41]:
Origin_Adress = 'Boulevard Piercot 29-25, 4000 Liège'
Origin_lat = 50.635451
Origin_lon = 5.570274

# get the venue
# radius = 1500
# LIMIT = 100
Origin_Result = list()
latitude = Origin_lat
longitude = Origin_lon
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
Origin_Result.append(requests.get(url).json())
# print Origin_Result

In [42]:
nearby_venues_list = list()
for results in Origin_Result:
    venues = results['response']['groups'][0]['items']
    # print ("venue", venues)
    nearby_venues = json_normalize(venues) # flatten JSON
    # print ('nearby_venues', nearby_venues)
    # filter columns
    filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
    # print 'toto'
    # print nearby_venues
    # check if results dataframe is empty 
    if nearby_venues.empty:
        print 'empty'
    else:
        nearby_venues =nearby_venues.loc[:, filtered_columns]

        # filter the category for each row
        nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

        # clean columns
        nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
        index = [index] * len(nearby_venues.index)
        nearby_venues['Cell_index'] = 'Origin'
        nearby_venues['Cell_Latitude'] = Origin_lat
        nearby_venues['Cell_Longitude'] = Origin_lon
    # Store results 
    nearby_venues_list.append(nearby_venues)
nearby_venues_list[0]

# let's group everything in one dataframe instead of a list of dataframe
df_Origin_Venue = pd.DataFrame() # nearby_venues_list[0]
for nv in nearby_venues_list:
    df_Origin_Venue = df_Origin_Venue.append(nv)

# reset index
df_Origin_Venue.reset_index(drop =True, inplace=True)


# lets reorganize the dataframe
#print nearby_venues_df['Postcode']
df_Origin_Venue = df_Origin_Venue.rename(columns={
    "name": "Venue",
    "lat": "Venue_Latitude",
    "lng": "Venue_Longitude",
    "categories": "Venue_Category"
})

# let's reorder 
df_Origin_Venue = df_Origin_Venue[['Cell_index', 'Cell_Latitude', 'Cell_Longitude', 'Venue', 'Venue_Category',
                                     'Venue_Latitude', 'Venue_Longitude']]




df_Origin_Venue.head(5)

,Cell_index,Cell_Latitude,Cell_Longitude,Venue,Venue_Category,Venue_Latitude,Venue_Longitude
0,Origin,50.635451,5.570274,The Huggy's Bar,Burger Joint,50.635638,5.567493
1,Origin,50.635451,5.570274,Parc d'Avroy,Park,50.633675,5.568395
2,Origin,50.635451,5.570274,Le Dernier Ragot,French Restaurant,50.639318,5.572951
3,Origin,50.635451,5.570274,Messieurs - Créateurs de croque monsieur,Restaurant,50.639354,5.568815
4,Origin,50.635451,5.570274,Franchi G.,Ice Cream Shop,50.639816,5.567891


In [43]:
# one hot encoding
df_Origin_Venue_onehot = pd.get_dummies(df_Origin_Venue[['Venue_Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
df_Origin_Venue_onehot['Cell_index'] = df_Origin_Venue['Cell_index'] 

# move neighborhood column to the first column
col_list =  list(df_Origin_Venue_onehot.columns)
col_list.pop(col_list.index('Cell_index'))
fixed_columns = ['Cell_index'] + col_list

df_Origin_Venue_onehot = df_Origin_Venue_onehot[fixed_columns]

df_Origin_Venue_onehot.head()

,Cell_index,Art Museum,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bar,Beer Bar,Bistro,Bookstore,...,Sandwich Place,Sculpture Garden,Shoe Store,Shopping Mall,Sushi Restaurant,Thai Restaurant,Theater,Vietnamese Restaurant,Wine Bar,Wine Shop
0,Origin,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Origin,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Origin,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Origin,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Origin,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
df_Origin_Venue_grouped = df_Origin_Venue_onehot.groupby('Cell_index').mean().reset_index()
df_Origin_Venue_grouped.head()



,Cell_index,Art Museum,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bar,Beer Bar,Bistro,Bookstore,...,Sandwich Place,Sculpture Garden,Shoe Store,Shopping Mall,Sushi Restaurant,Thai Restaurant,Theater,Vietnamese Restaurant,Wine Bar,Wine Shop
0,Origin,0.01,0.01,0.01,0.01,0.01,0.05,0.01,0.02,0.02,...,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.02,0.02,0.01


We will certainly have less feature that in the Houston case (much narrow region)
If we want to predict using the defined model for Houston, we have to have the same features
Conversely, they might have category that do not exist in the destination town. In that case, we will simply remove them. They do not exist in the destination town. Regarding those feature, there is no interest to use them because there are an equal factor of disimilarity for each destination cells. 


In [45]:
# add missing features
origin_col = df_Origin_Venue_grouped.columns
destin_col = df_Houston_Cell_Venue_grouped.columns
for col in df_Houston_Cell_Venue_grouped.columns:
    if not col in origin_col:
        df_Origin_Venue_grouped[col] = 0
        
        
# remove unique feature not found in destination
NotExistingCategoryInDestination =  []
df_Origin_Venue_grouped_filt = pd.DataFrame(df_Origin_Venue_grouped)
for col in origin_col:
    if not col in destin_col:
        NotExistingCategoryInDestination.append(col)
        df_Origin_Venue_grouped_filt.drop(col, 1, inplace = True)
df_Origin_Venue_grouped_filt.head()

,Cell_index,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bar,Beer Bar,Bistro,Bookstore,Boutique,...,Warehouse Store,Water Park,Weight Loss Center,Whisky Bar,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo,Zoo Exhibit
0,Origin,0.01,0.01,0.01,0.01,0.05,0.01,0.02,0.02,0.01,...,0,0,0,0,0,0,0,0,0,0


Note: the following venues don't exist in the destination town. So sad.

In [46]:
print NotExistingCategoryInDestination

[u'Art Museum', u'Brasserie', u'Bridge', u'Butcher', u'Friterie', u'Moroccan Restaurant', u'Opera House']


Get the corresponding cluster

In [47]:
# # number of cluster, K
# K = 10
# 
df_Origin_clustering = df_Origin_Venue_grouped_filt.drop('Cell_index', 1)
# 
# # run k-means clustering
# kmeans = KMeans(n_clusters=K, random_state=0).fit(df_Houston_Cell_clustering)
# 
# # check cluster labels generated for each row in the dataframe
# kmeans.labels_[0:100] 
# help(kmeans)
Origin_Cluster = kmeans.predict(df_Origin_clustering)
print 'the origin town is closer to cluster n°', Origin_Cluster
 

the origin town is closer to cluster n° [1]


## Houston : destination refining
Based on the category of venue in origin town, let's see what are the closest cells 

In [447]:
# filter cells to the one of hte same cluster
df_Houston_Cell_venues_ClusterOrigin = pd.DataFrame(df_Houston_Cell_venues_merged)
df_Houston_Cell_venues_ClusterOrigin
# Do the filtering there is something like df_Houston_Cell_venues_ClusterOrigin = df_Houston_Cell_venues_ClusterOrigin['Cluster'] == 8
# create map
map_clusters = folium.Map(location=[Houston_Coord_lat, Houston_Coord_lon], zoom_start=11)


# set color scheme for the clusters
x = np.arange(K)
ys = [i + x + (i*x)**2 for i in range(K)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_Houston_Cell_venues_merged['Cell_Latitude'], df_Houston_Cell_venues_merged['Cell_Longitude'], df_Houston_Cell_venues_merged['Cell_index'], df_Houston_Cell_venues_merged['Cluster Labels']):
    if cluster == Origin_Cluster:
        label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[cluster-1],
            fill=True,
            fill_color=rainbow[cluster-1],
            fill_opacity=0.7).add_to(map_clusters)
map_clusters

## 5. Refine according to House prices

### 5.1 Load the data
Note: House prices per ft² provided by RedFin database.. Thank to them.  
Note2: House prices per ft² provided by zipcode

In [192]:
HousePrices_FilePath = 'Houston_RentingPrice_RedFin.csv'

df_housePrice = pd.read_csv(HousePrices_FilePath)

# rename columns so it is easier to manipulate
df_housePrice.rename(columns={ df_housePrice.columns[0]: "zipcode" , df_housePrice.columns[1]: "price_sqft" }, inplace = True)
df_housePrice.head()


,zipcode,price_sqft
0,77459,111
1,77477,100
2,77099,89
3,77072,89
4,77042,109


### 5.2 Clean data set
Missing dataset are labeled with X or '-
Replace them by the averaged prices

In [231]:
# CHECK DATA
# print df_housePrice
# pd.options.display.max_rows=12
#print df_housePrice.price_sqft.mean()

# Get average of numeric values
df_housePrice.price_sqft = df_housePrice.price_sqft.astype(str)
DS_ValidPrice = df_housePrice[df_housePrice.price_sqft.str.isdigit()].price_sqft
DS_ValidPrice = DS_ValidPrice.astype(float)

price_average = DS_ValidPrice.mean()
print 'average price', price_average

# Replace no numeric values
df_housePriceFilt = pd.DataFrame(df_housePrice)
# value to replace
ValueToReplace = [not i for i in df_housePriceFilt.price_sqft.str.isdigit()]

df_housePriceFilt.loc[ValueToReplace, 'price_sqft' ] = price_average
df_housePriceFilt.head()

#pd.options.display.max_rows=12
#

average price 128.098360656


,zipcode,price_sqft
0,77459,111
1,77477,100
2,77099,89
3,77072,89
4,77042,109


### 5.3 Get zip code for each cell of our dataframe 
For that, we will use python library **geopy.geocoders** that allows to get coordinate of adress or adress of coordinate

In [106]:
# Get Zip code for each cell
# import time

# locator = Nominatim(user_agent="myGeocoder")
# location = locator.reverse(coordinates)
# z = location.raw['address']['postcode'][0:4]

locator = Nominatim(user_agent="myGeocoder")
df_Houston_Cell_venues_merged_price = df_Houston_Cell_venues_merged
for index, row in df_Houston_Cell_venues_merged_price.iterrows():
  # print row['Cell_Latitude'], row['Cell_Longitude']
  coordinates = (row['Cell_Latitude'] , row['Cell_Longitude'])
  location = locator.reverse(coordinates)
  zipcode = location.raw['address']['postcode']
  df_Houston_Cell_venues_merged_price.at[index,'zipcode'] = zipcode[0:5]
  time.sleep(0.5) # otherwise, service timeout
df_Houston_Cell_venues_merged_price.head()

,Cell_Latitude,Cell_Longitude,Cell_index,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Zip,zipcode
0,29.601804,-95.561416,0,2,Mexican Restaurant,Pharmacy,Coffee Shop,Storage Facility,Fast Food Restaurant,Burger Joint,Camera Store,Gas Station,Sandwich Place,Juice Bar,77044,77459
1,29.637868,-95.562329,1,1,Mexican Restaurant,Sandwich Place,Seafood Restaurant,Deli / Bodega,Video Store,Bar,Comfort Food Restaurant,Pharmacy,Fried Chicken Joint,Food,77044,77477
2,29.673933,-95.563244,2,2,Fast Food Restaurant,Mobile Phone Shop,Mexican Restaurant,Gas Station,Video Store,Asian Restaurant,Storage Facility,Other Repair Shop,Fried Chicken Joint,Motel,77044,77099
3,29.709996,-95.564160,3,1,Chinese Restaurant,Asian Restaurant,Bubble Tea Shop,Vietnamese Restaurant,Korean Restaurant,Bakery,Hotel,Ice Cream Shop,Juice Bar,Fast Food Restaurant,77044,77072
4,29.746060,-95.565077,4,1,Hotel,Sandwich Place,Grocery Store,Bakery,Chinese Restaurant,Mexican Restaurant,Pizza Place,Discount Store,Japanese Restaurant,Breakfast Spot,77044,77042


### 5.4 Merge with price per ft_sq

In [232]:
df_Houston_Cell_venues_merged_price.zipcode = df_Houston_Cell_venues_merged_price.zipcode.astype(int)
df_housePriceFilt.zipcode = df_housePriceFilt.zipcode.astype(int)
df_housePriceFilt.price_sqft = df_housePriceFilt.price_sqft.astype(float)

# check if columns exist, if yes, drop it before merging
if 'zipcode' in df_Houston_Cell_venues_merged_price:
  df_Houston_Cell_venues_merged_price.drop('price_sqft', axis=1, inplace = True)

df_Houston_Cell_venues_merged_price = df_Houston_Cell_venues_merged_price.join(df_housePriceFilt.set_index('zipcode'), on='zipcode')
df_Houston_Cell_venues_merged_price.head()
# 

,Cell_Latitude,Cell_Longitude,Cell_index,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,zipcode,price_sqft
0,29.601804,-95.561416,0,2,Mexican Restaurant,Pharmacy,Coffee Shop,Storage Facility,Fast Food Restaurant,Burger Joint,Camera Store,Gas Station,Sandwich Place,Juice Bar,77459,111.0
1,29.637868,-95.562329,1,1,Mexican Restaurant,Sandwich Place,Seafood Restaurant,Deli / Bodega,Video Store,Bar,Comfort Food Restaurant,Pharmacy,Fried Chicken Joint,Food,77477,100.0
2,29.673933,-95.563244,2,2,Fast Food Restaurant,Mobile Phone Shop,Mexican Restaurant,Gas Station,Video Store,Asian Restaurant,Storage Facility,Other Repair Shop,Fried Chicken Joint,Motel,77099,89.0
3,29.709996,-95.564160,3,1,Chinese Restaurant,Asian Restaurant,Bubble Tea Shop,Vietnamese Restaurant,Korean Restaurant,Bakery,Hotel,Ice Cream Shop,Juice Bar,Fast Food Restaurant,77072,89.0
4,29.746060,-95.565077,4,1,Hotel,Sandwich Place,Grocery Store,Bakery,Chinese Restaurant,Mexican Restaurant,Pizza Place,Discount Store,Japanese Restaurant,Breakfast Spot,77042,109.0


### 5.5 Display the map of selected area (in 4.) but broken down by price

In [268]:
# filter cells to the one of hte same cluster
#df_Houston_Cell_venues_ClusterOrigin = pd.DataFrame(df_Houston_Cell_venues_merged)
df_Houston_Cell_venues_merged_price_SelectedCluster = df_Houston_Cell_venues_merged_price[df_Houston_Cell_venues_merged_price['Cluster Labels'] == Origin_Cluster[0]]

df_Houston_Cell_venues_merged_price_SelectedCluster.head()
# Do the filtering there is something like df_Houston_Cell_venues_ClusterOrigin = df_Houston_Cell_venues_ClusterOrigin['Cluster'] == 8
# create map

# check price ranges
min_price = df_Houston_Cell_venues_merged_price_SelectedCluster.price_sqft.min()
max_price = df_Houston_Cell_venues_merged_price_SelectedCluster.price_sqft.max()
print 'min price is: ', min_price
print 'max price is: ', max_price

# split price in 10 price category
nb_bin = 10
print type(max_price)
Price_Bin = [min_price + i * (max_price - min_price) / (nb_bin*1.0) for i in range(0, 11) ]

# next line of code is just to make sure the most expensive one is included in the selection as it price as to be < last price
Price_Bin[len(Price_Bin)-1] += 1 
# add the price_cat in the df
for i in range(0,nb_bin):
    print 'cat ', i, ' : ', Price_Bin[i] ,' - ' , Price_Bin[i+1]
    ValueToReplace = [ p >= Price_Bin[i] and p < Price_Bin[i+1] for p in df_Houston_Cell_venues_merged_price_SelectedCluster.price_sqft]
    df_Houston_Cell_venues_merged_price_SelectedCluster.loc[ValueToReplace, 'price_cat' ] = int(i)    
df_Houston_Cell_venues_merged_price_SelectedCluster.price_cat = df_Houston_Cell_venues_merged_price_SelectedCluster.price_cat.astype(int, inplace=True)   
pd.options.display.max_rows=1000
df_Houston_Cell_venues_merged_price_SelectedCluster.head(5)

min price is:  87.0
max price is:  360.0
<type 'numpy.float64'>
cat  0  :  87.0  -  114.3
cat  1  :  114.3  -  141.6
cat  2  :  141.6  -  168.9
cat  3  :  168.9  -  196.2
cat  4  :  196.2  -  223.5
cat  5  :  223.5  -  250.8
cat  6  :  250.8  -  278.1
cat  7  :  278.1  -  305.4
cat  8  :  305.4  -  332.7
cat  9  :  332.7  -  361.0


,Cell_Latitude,Cell_Longitude,Cell_index,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,zipcode,price_sqft,price_cat
1,29.637868,-95.562329,1,1,Mexican Restaurant,Sandwich Place,Seafood Restaurant,Deli / Bodega,Video Store,Bar,Comfort Food Restaurant,Pharmacy,Fried Chicken Joint,Food,77477,100.0,0
3,29.709996,-95.564160,3,1,Chinese Restaurant,Asian Restaurant,Bubble Tea Shop,Vietnamese Restaurant,Korean Restaurant,Bakery,Hotel,Ice Cream Shop,Juice Bar,Fast Food Restaurant,77072,89.0,0
4,29.746060,-95.565077,4,1,Hotel,Sandwich Place,Grocery Store,Bakery,Chinese Restaurant,Mexican Restaurant,Pizza Place,Discount Store,Japanese Restaurant,Breakfast Spot,77042,109.0,0
5,29.782123,-95.565997,5,1,Clothing Store,Mexican Restaurant,Burger Joint,Coffee Shop,American Restaurant,Furniture / Home Store,Cosmetics Shop,Sandwich Place,Seafood Restaurant,Steakhouse,77079,145.0,2
7,29.854250,-95.567841,7,1,Furniture / Home Store,Convenience Store,Burger Joint,Donut Shop,Auto Garage,Thai Restaurant,Grocery Store,Restaurant,Gas Station,Music Store,77041,109.0,0


In [275]:
# generate the folium map

map_clusters = folium.Map(location=[Houston_Coord_lat, Houston_Coord_lon], zoom_start=11)


# set color scheme for the price ranges
x = np.arange(nb_bin)
ys = [i + x + (i*x)**2 for i in range(K)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster, price_sqft, price_cat in zip(df_Houston_Cell_venues_merged_price_SelectedCluster['Cell_Latitude'], 
                                              df_Houston_Cell_venues_merged_price_SelectedCluster['Cell_Longitude'], 
                                              df_Houston_Cell_venues_merged_price_SelectedCluster['Cell_index'], 
                                              df_Houston_Cell_venues_merged_price_SelectedCluster['Cluster Labels'],
                                              df_Houston_Cell_venues_merged_price_SelectedCluster['price_sqft'],
                                              df_Houston_Cell_venues_merged_price_SelectedCluster['price_cat']):
    if cluster == Origin_Cluster:
        # note: not possible to do newline anymore with <br> ? \n doesn't work either
        label = folium.Popup(str(poi) + ' Cluster ' + str(cluster) +
                             '- \n  Price per sqft ' + str(price_sqft), parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[price_cat],
            fill=True,
            fill_color=rainbow[price_cat],
            fill_opacity=0.7).add_to(map_clusters)

        
map_clusters

## 6. Refine according to Criminal data
Now that we have find the location that are the most similar to our origin address and that we have sorted them into price range, we will check criminality rates for each cells and just keep the best ones
Best ones are the one that :
- for a given price, there is not other place with less criminal records
- for a given criminal records score, there is no other place cheaper

Note: criminal offense have been converted to a score following that rules:
    1. no physical damage (theft, fraud) = 1 point
    2. simple assault	= 2 points
    3. aggravated assault = 3 points
    4. Rape = 5 point 
    5. Homicide = 10 point
Lowest score = better

### 6.1 Get criminal records

In [375]:
Criminality_FilePath = 'NIBRSPublicView.Jan1-Jan31-FINAL.csv'

df_Criminal_raw = pd.read_csv(Criminality_FilePath)

# rename columns so it is easier to manipulate
df_Criminal_raw.head()

,Incident,Occurrence Date,Occurrence Hour,NIBRS Class,NIBRSDescription,Offense Count,Beat,Premise,Block Range,StreetName,Street Type,Suffix,ZIP Code
0,8220,01/01/2020,0,23G,Theft of motor vehicle parts or accessory,1,8C50,"Residence, Home (Includes Apartment)",9311,BELLA PINE,CT,NaN,77078
1,23020,01/01/2020,0,290,"Destruction, damage, vandalism",1,11H30,"Residence, Home (Includes Apartment)",8064,LENORE,NaN,NaN,77017
2,24120,01/01/2020,0,13B,Simple assault,1,17E10,"Residence, Home (Includes Apartment)",5930,DASHWOOD,DR,NaN,77081
3,27120,01/01/2020,0,290,"Destruction, damage, vandalism",1,14D30,"Residence, Home (Includes Apartment)",5218,KENILWOOD,DR,NaN,77033
4,29320,01/01/2020,0,290,"Destruction, damage, vandalism",1,20G30,"Other, Unknown",2851,WALLINGFORD,DR,NaN,77042


### 6.2 Clean dataset 
keep columns of interest :  
- NIBRSDescription
- Block Range
- StreetName
- ZIP Code

Remove Na columns
Rename columns

In [376]:
# Note, should I compare using zip code or locate most nearby cell using ( in that case, do it on all cells !)

# keep columns of interest
df_Criminal = df_Criminal_raw[['NIBRSDescription', 'Block Range', 'StreetName', 'ZIP Code']]
df_Criminal.head()

# remove NAN
df_Criminal.dropna(axis=0, inplace=True)

# Rename columns
df_Criminal.rename(columns={ df_Criminal.columns[0]: "description" ,
                             df_Criminal.columns[1]: "block",
                             df_Criminal.columns[2]: "street",
                             df_Criminal.columns[3]: "zipcode"}, inplace = True)


df_Criminal.head()

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,description,block,street,zipcode
0,Theft of motor vehicle parts or accessory,9311,BELLA PINE,77078
1,"Destruction, damage, vandalism",8064,LENORE,77017
2,Simple assault,5930,DASHWOOD,77081
3,"Destruction, damage, vandalism",5218,KENILWOOD,77033
4,"Destruction, damage, vandalism",2851,WALLINGFORD,77042


### 6.3 Give a score to each offense entry

In [377]:
# Get unique values of offense:
df_Criminal.description.unique()

array(['Theft of motor vehicle parts or accessory',
       'Destruction, damage, vandalism', 'Simple assault',
       'Theft from motor vehicle', 'Drug, narcotic violations',
       'Aggravated Assault', 'Intimidation', 'Robbery', 'Forcible rape',
       'Motor vehicle theft', 'Credit card, ATM fraud', 'Identify theft',
       'Drug equipment violations', 'Murder, non-negligent',
       'All other larceny', 'Shoplifting', 'Arson',
       'Weapon law violations', 'Pocket-picking',
       'Burglary, Breaking and Entering', 'Forcible sodomy',
       'Counterfeiting, forgery', 'Pornographs, obscene material',
       'Theft from building', 'Forcible fondling', 'Purse-snatching',
       'Bribery', 'Extortion, Blackmail', 'Embezzlement',
       'False pretenses, swindle', 'Human Trafficking/Commercial Sex Act',
       'Animal Cruelty', 'Kidnapping, abduction',
       'From coin-operated machine or device', 'Statutory rape',
       'Stolen property offenses', 'Prostitution', 'Impersonation',
 

In [378]:
# Define score:
# 1. no physical damage (theft, fraud) = 1 point
# 2. simple assault    = 2 points
# 3. aggravated assault = 3 points
# 4. Rape = 5 point 
# 5. Homicide = 10 point
Scoring = {'Theft of motor vehicle parts or accessory' : 1,
           'Destruction, damage, vandalism':1, 
           'Simple assault':2,
           'Theft from motor vehicle':1, 
           'Drug, narcotic violations':1,
           'Aggravated Assault':3, 
           'Intimidation':2, 
           'Robbery':1, 
           'Forcible rape':5,
           'Motor vehicle theft':1, 
           'Credit card, ATM fraud':1, 
           'Identify theft':1,
           'Drug equipment violations':1, 
           'Murder, non-negligent':10,
           'All other larceny':1, 
           'Shoplifting':1, 
           'Arson':2,
           'Weapon law violations':1, 
           'Pocket-picking':1,
           'Burglary, Breaking and Entering':1,
           'Forcible sodomy':5,
           'Counterfeiting, forgery':1, 
           'Pornographs, obscene material':5,
           'Theft from building':1, 
           'Forcible fondling':1, 
           'Purse-snatching':2,
           'Bribery':1, 
           'Extortion, Blackmail':2, 
           'Embezzlement':1,
           'False pretenses, swindle':1,
           'Human Trafficking/Commercial Sex Act':5,
           'Animal Cruelty':2, 
           'Kidnapping, abduction':5,
           'From coin-operated machine or device':1, 
           'Statutory rape':5,
           'Stolen property offenses':1,
           'Prostitution':1, 
           'Impersonation':1,
           'Wire fraud':1, 
           'Hacking/Computer Invasion':1,
           'Purchasing prostitution':1, 
           'Incest':5,
           'Assisting or promoting prostitution':1,
           'Sexual assault with an object':5}



In [298]:
# replace(self, to_replace=None, value=None, inplace=False, limit=None, regex=False, method='pad')
df_Criminal['score'] = df_Criminal.description.replace(Scoring)
df_Criminal.head()
#for key, value in Scoring.items():
#  df_Criminal['score'] = df_Criminal.description.replace(key, value)

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,description,block,street,zipcode,score
0,Theft of motor vehicle parts or accessory,9311,BELLA PINE,77078,1
1,"Destruction, damage, vandalism",8064,LENORE,77017,1
2,Simple assault,5930,DASHWOOD,77081,2
3,"Destruction, damage, vandalism",5218,KENILWOOD,77033,1
4,"Destruction, damage, vandalism",2851,WALLINGFORD,77042,1


### 6.4 Assign offense to cell then calculate cell total score

In [383]:
# test wiht one adress
block = 9311
street = 'BELLA PINE'
zipcode = '77078'
address = str(block) + ', ' + street + ', ' + zipcode
print address

locator = Nominatim(user_agent="myGeocoder")

location = locator.geocode(address)

print((location.latitude, location.longitude))


# First, get coordinate for each offense
df_Criminal_cord = df_Criminal
for index, row in df_Criminal_cord.iterrows():
  #if index < 10:
    address = str(row.block) + ', ' + row.street + ', ' + row.zipcode
    # print address
    location = None
    try:
      location = locator.geocode(address)
    except:
      time.sleep(2)
      print("An exception occurred at ", index, ", trying agian") 
      location = locator.geocode(address)
    if location is None:
      df_Criminal_cord.at[index,'latitude'] = float('nan')
      df_Criminal_cord.at[index,'longitude'] = float('nan')
    else:
      df_Criminal_cord.at[index,'latitude'] = location.latitude
      df_Criminal_cord.at[index,'longitude'] = location.longitude
    #time.sleep(0.5) # otherwise, service timeout

df_Criminal_cord.head(10)

9311, BELLA PINE, 77078
(29.856284897959185, -95.25162830612244)
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exceptio

An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An excepti

An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An excepti

An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An excepti

An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An excepti

...

An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An excepti

An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An excepti

An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An excepti

An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An excepti

,description,block,street,zipcode,latitude,longitude
0,Theft of motor vehicle parts or accessory,9311,BELLA PINE,77078,29.856285,-95.251628
1,"Destruction, damage, vandalism",8064,LENORE,77017,29.676495,-95.263450
2,Simple assault,5930,DASHWOOD,77081,29.707437,-95.485750
3,"Destruction, damage, vandalism",5218,KENILWOOD,77033,29.672524,-95.344034
4,"Destruction, damage, vandalism",2851,WALLINGFORD,77042,29.732533,-95.563735
5,"Destruction, damage, vandalism",16410,SKEET,77489,29.588750,-95.510778
6,Theft from motor vehicle,400,80TH,77012,29.731449,-95.281358
7,Simple assault,2434,LIDSTONE,77023,29.715633,-95.321752
8,Simple assault,8115,PENROD,77028,29.825891,-95.278233
9,"Destruction, damage, vandalism",16214,LAZY RIDGE,77053,NaN,NaN


In [386]:
# remove row without value
df_Criminal_cord_to_cell = df_Criminal_cord.dropna(axis=0)
df_Criminal_cord_to_cell.head(10)

# calculate distance for one offense
cell_index = df_Houston_Cell_venues_merged_price.Cell_index

# loop
for index, row in df_Criminal_cord_to_cell.iterrows():
  dist_lat = df_Houston_Cell_venues_merged_price.Cell_Latitude - row['latitude']
  dist_lon = df_Houston_Cell_venues_merged_price.Cell_Longitude - row['longitude']
  dist_lat = np.array(dist_lat)
  dist_lon = np.array(dist_lon)
  dist_lat = [i**2 for i in dist_lat]
  dist_lon = [i**2 for i in dist_lon]
  dist_dist = np.array([(i+j)**0.5 for i, j in zip(dist_lat, dist_lon) ])
  df_Criminal_cord_to_cell.at[index,'Cell_index'] = cell_index[dist_dist.argmin()]


#df_Criminal_cord_to_cell.Cell_index.astype(int)
df_Criminal_cord_to_cell.head(15)


,description,block,street,zipcode,latitude,longitude,Cell_index
0,Theft of motor vehicle parts or accessory,9311,BELLA PINE,77078,29.856285,-95.251628,77.0
1,"Destruction, damage, vandalism",8064,LENORE,77017,29.676495,-95.263450,72.0
2,Simple assault,5930,DASHWOOD,77081,29.707437,-95.485750,23.0
3,"Destruction, damage, vandalism",5218,KENILWOOD,77033,29.672524,-95.344034,52.0
4,"Destruction, damage, vandalism",2851,WALLINGFORD,77042,29.732533,-95.563735,4.0
5,"Destruction, damage, vandalism",16410,SKEET,77489,29.588750,-95.510778,10.0
6,Theft from motor vehicle,400,80TH,77012,29.731449,-95.281358,73.0
7,Simple assault,2434,LIDSTONE,77023,29.715633,-95.321752,63.0
8,Simple assault,8115,PENROD,77028,29.825891,-95.278233,76.0
10,"Drug, narcotic violations",1415,GREENS,77067,30.010478,-95.477270,29.0


In [390]:
df_Criminal_cord_to_cell['score'] = df_Criminal_cord_to_cell.description.replace(Scoring)
df_Criminal_cord_to_cell.head()

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,description,block,street,zipcode,latitude,longitude,Cell_index,score
0,Theft of motor vehicle parts or accessory,9311,BELLA PINE,77078,29.856285,-95.251628,77.0,1
1,"Destruction, damage, vandalism",8064,LENORE,77017,29.676495,-95.263450,72.0,1
2,Simple assault,5930,DASHWOOD,77081,29.707437,-95.485750,23.0,2
3,"Destruction, damage, vandalism",5218,KENILWOOD,77033,29.672524,-95.344034,52.0,1
4,"Destruction, damage, vandalism",2851,WALLINGFORD,77042,29.732533,-95.563735,4.0,1


In [413]:
# Get total score for each cell index
df_Criminal_cord_to_cell_grouped = df_Criminal_cord_to_cell.groupby('Cell_index').sum().reset_index()
# drop lattitude and longitude that are no longer needed
df_Criminal_cord_to_cell_grouped = df_Criminal_cord_to_cell_grouped.drop('latitude', axis=1)
df_Criminal_cord_to_cell_grouped = df_Criminal_cord_to_cell_grouped.drop('longitude', axis=1)


print df_Criminal_cord_to_cell_grouped.shape
# some cells may have no crime record for some reason, in such a case, they don't exist in the data fram
# add them
df_Criminal_cord_to_cell_grouped.Cell_index = df_Criminal_cord_to_cell_grouped.Cell_index.astype(int)
for i in range(0, 100):
   # if not i in df_Criminal_cord_to_cell_grouped.Cell_index: # note htat seems to not see missing values !!!
    if not any(df_Criminal_cord_to_cell_grouped.Cell_index.isin([i])):
        print i, ' is missing'
        line = pd.DataFrame({"Cell_index": i, "score": 0}, index=[i+0.5])
        df_Criminal_cord_to_cell_grouped = df_Criminal_cord_to_cell_grouped.append(line, ignore_index=False)
        df_Criminal_cord_to_cell_grouped = df_Criminal_cord_to_cell_grouped.sort_index().reset_index(drop=True)

df_Criminal_cord_to_cell_grouped.head(100)

(85, 2)
8  is missing
18  is missing
58  is missing
60  is missing
82  is missing
83  is missing
84  is missing
86  is missing
87  is missing
91  is missing
92  is missing
95  is missing
96  is missing
97  is missing
99  is missing


,Cell_index,score
0,0,1
1,1,12
2,2,51
3,3,44
4,4,1900
5,5,16
6,6,12
7,7,1
8,9,764
9,8,0


### 6.5 Merge Selected cluster, price and criminality, then only retains optimum price-criminity cells
Let call the dataframe df_HCVMPCSC because it starts to be loong name, that stand for  
df_Houston_Cell_Venues_Merged_Price_Criminality_SelectedCluster

In [425]:
#df_Houston_Cell_venues_merged_price_SelectedCluster

#df_HCVMPCSC = df_Houston_Cell_venues_merged_price_SelectedCluster.join(df_Criminal_cord_to_cell_grouped.set_index('Cell_index'), on='Cell_index')

df_HCVMPCSC = df_Houston_Cell_venues_merged_price_SelectedCluster

for index, row in df_HCVMPCSC.iterrows():
  score =  df_Criminal_cord_to_cell_grouped.loc[df_Criminal_cord_to_cell_grouped['Cell_index'] == row['Cell_index']].score.to_string(index=False)
  df_HCVMPCSC.at[index,'criminality'] = score

df_HCVMPCSC.head(100)

,Cell_Latitude,Cell_Longitude,Cell_index,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,zipcode,price_sqft,price_cat,criminality
1,29.637868,-95.562329,1,1,Mexican Restaurant,Sandwich Place,Seafood Restaurant,Deli / Bodega,Video Store,Bar,Comfort Food Restaurant,Pharmacy,Fried Chicken Joint,Food,77477,100.000000,0,12.0
3,29.709996,-95.564160,3,1,Chinese Restaurant,Asian Restaurant,Bubble Tea Shop,Vietnamese Restaurant,Korean Restaurant,Bakery,Hotel,Ice Cream Shop,Juice Bar,Fast Food Restaurant,77072,89.000000,0,44.0
4,29.746060,-95.565077,4,1,Hotel,Sandwich Place,Grocery Store,Bakery,Chinese Restaurant,Mexican Restaurant,Pizza Place,Discount Store,Japanese Restaurant,Breakfast Spot,77042,109.000000,0,1900.0
5,29.782123,-95.565997,5,1,Clothing Store,Mexican Restaurant,Burger Joint,Coffee Shop,American Restaurant,Furniture / Home Store,Cosmetics Shop,Sandwich Place,Seafood Restaurant,Steakhouse,77079,145.000000,2,16.0
7,29.854250,-95.567841,7,1,Furniture / Home Store,Convenience Store,Burger Joint,Donut Shop,Auto Garage,Thai Restaurant,Grocery Store,Restaurant,Gas Station,Music Store,77041,109.000000,0,1.0
8,29.890312,-95.568765,8,1,Mexican Restaurant,Chinese Restaurant,Sandwich Place,Sporting Goods Shop,Liquor Store,Fried Chicken Joint,Taco Place,Food Service,Sports Bar,Burger Joint,77040,106.000000,0,0.0
9,29.926375,-95.569691,9,1,Spanish Restaurant,Mexican Restaurant,Pizza Place,Vietnamese Restaurant,Football Stadium,Automotive Shop,Taco Place,Trail,Big Box Store,Deli / Bodega,77064,95.000000,0,764.0
12,29.674725,-95.521945,12,1,Discount Store,Sandwich Place,Pharmacy,Pizza Place,Clothing Store,Pet Store,Trade School,Food,Food & Drink Shop,Thrift / Vintage Store,77074,111.000000,0,26.0
13,29.710790,-95.522847,13,1,Furniture / Home Store,Convenience Store,Fast Food Restaurant,Electronics Store,Shoe Store,Gas Station,Mexican Restaurant,Pizza Place,Sandwich Place,Latin American Restaurant,77036,93.000000,0,20.0
14,29.746855,-95.523750,14,1,Pizza Place,Rental Car Location,Bank,Coffee Shop,Pharmacy,Fast Food Restaurant,Mobile Phone Shop,Mexican Restaurant,Gas Station,American Restaurant,77063,138.000000,1,307.0


In [442]:
# now delete all duplicate of price category to retain the lowest criminality score
print df_HCVMPCSC.price_cat.unique()

df_HCVMPCSC_final = pd.DataFrame()

for PriceCat in df_HCVMPCSC.price_cat.unique():
  df_HCVMPCSC_x = df_HCVMPCSC[df_HCVMPCSC.price_cat == PriceCat].sort_values(['criminality'], ascending=True)
  df_HCVMPCSC_x = df_HCVMPCSC_x.drop_duplicates(subset=['price_cat'], keep='first')

  df_HCVMPCSC_final = df_HCVMPCSC_final.append(df_HCVMPCSC_x)
df_HCVMPCSC_final.head(10)

[0 2 1 6 3 9 5 4]


,Cell_Latitude,Cell_Longitude,Cell_index,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,zipcode,price_sqft,price_cat,criminality
99,29.933116,-95.197018,99,1,Fast Food Restaurant,Asian Restaurant,Pizza Place,Department Store,Mexican Restaurant,Burger Joint,Pharmacy,Sushi Restaurant,Bank,Fried Chicken Joint,77044,98.0,0,0.0
5,29.782123,-95.565997,5,1,Clothing Store,Mexican Restaurant,Burger Joint,Coffee Shop,American Restaurant,Furniture / Home Store,Cosmetics Shop,Sandwich Place,Seafood Restaurant,Steakhouse,77079,145.0,2,16.0
42,29.677027,-95.398041,42,1,BBQ Joint,Rental Car Location,Gas Station,General Entertainment,Bar,Pizza Place,Fast Food Restaurant,Moving Target,Smoke Shop,Theme Park Ride / Attraction,77054,135.0,1,4.0
25,29.783703,-95.483310,25,1,Spa,Donut Shop,Italian Restaurant,Sushi Restaurant,Liquor Store,Sandwich Place,Video Store,Gas Station,Pet Store,Pizza Place,77024,255.0,6,2.0
45,29.785231,-95.400617,45,1,Mexican Restaurant,Cajun / Creole Restaurant,Coffee Shop,Park,Spa,American Restaurant,Bar,Ice Cream Shop,Italian Restaurant,Juice Bar,77007,187.0,3,17.0
33,29.712340,-95.440216,33,1,Pizza Place,Gym,Mexican Restaurant,Bakery,American Restaurant,Sandwich Place,Coffee Shop,Italian Restaurant,Convenience Store,Gym / Fitness Center,77005,360.0,9,500.0
34,29.748407,-95.441090,34,1,Furniture / Home Store,Women's Store,Hotel,Coffee Shop,Clothing Store,Sandwich Place,Cosmetics Shop,Bank,French Restaurant,Shopping Mall,77019,229.0,5,3096.0
41,29.640958,-95.397185,41,1,Park,Candy Store,Golf Course,Museum,Athletics & Sports,Sandwich Place,Soccer Field,Rugby Stadium,Electronics Store,Donut Shop,77030,203.0,4,4.0


In [427]:
# now delete all row where it exist a criminality score lower for a lower price (=bad choice to move in)

Criminality data where only based on a month because it is a lot of entry (~20k) and the API (right) doesn't allows to make too many request. Some cells doesn't have criminality records 
Based on that data, the best cell to move in is the one with the cell_index 99 : absolute BEST PRICE and best criminality score

# 7 show the final the map

In [444]:
# generate the folium map

map_clusters = folium.Map(location=[Houston_Coord_lat, Houston_Coord_lon], zoom_start=11)


# set color scheme for the price ranges
x = np.arange(nb_bin)
ys = [i + x + (i*x)**2 for i in range(K)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster, price_sqft, price_cat, crime in zip(df_HCVMPCSC_final['Cell_Latitude'], 
                                              df_HCVMPCSC_final['Cell_Longitude'], 
                                              df_HCVMPCSC_final['Cell_index'], 
                                              df_HCVMPCSC_final['Cluster Labels'],
                                              df_HCVMPCSC_final['price_sqft'],
                                              df_HCVMPCSC_final['price_cat'],
                                              df_HCVMPCSC_final['criminality']):
    if cluster == Origin_Cluster:
        # note: not possible to do newline anymore with <br> ? \n doesn't work either
        label = folium.Popup(str(poi) + ' Cluster ' + str(cluster) +
                             '- \n  Price per sqft ' + str(price_sqft) +
                             '- \n Criminality score' + str(crime), parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[price_cat],
            fill=True,
            fill_color=rainbow[price_cat],
            fill_opacity=0.7).add_to(map_clusters)

        
map_clusters